In [1]:
#!pip install torchmetrics

In [3]:
import requests
import os
import logging
import gdown
import random

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
from torchvision.datasets import VOCSegmentation
import torchmetrics
import torchvision
import albumentations as A

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import cv2
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as T

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from scipy.io import loadmat
from sklearn.manifold import TSNE
from torchmetrics.functional.classification import multiclass_accuracy
from torchmetrics.classification import MulticlassF1Score, JaccardIndex, MulticlassPrecision, MulticlassRecall, MulticlassAveragePrecision
import pandas as pd

In [4]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
logging.basicConfig(level=logging.ERROR)

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


# Ensemble Strategy

In [5]:
prepro = False
BATCH_SIZE = 16

In [6]:
ROOT_DIR = '../Datasets/ocular-disease-recognition-odir5k'

In [7]:
IMG_FOLDER = ROOT_DIR + ""
HIST_IMG_FOLDER = ROOT_DIR + ""
normal_eff_model = "Saved_Models/"
grey_eff_model = "Saved_Models/"

In [8]:
if(prepro == True):
    IMG_FOLDER = ROOT_DIR + '/inception_preprocessed_images'
    HIST_IMG_FOLDER = ROOT_DIR + '/inception_histeq_preprocessed_images'
    normal_eff_model += "eff_b3_normal_preproc.pt"
    grey_eff_model += "eff_b3_grey_preproc.pt"
else:
    IMG_FOLDER = ROOT_DIR + '/preprocessed_images'
    HIST_IMG_FOLDER = ROOT_DIR + '/preprocessed_histeq_images'
    normal_eff_model +="eff_b3_normal.pt"
    grey_eff_model += "eff_b3_grey.pt"

CSV_PATH = ROOT_DIR + '/dataset_single_eye.csv'
TEST_CSV = ROOT_DIR + '/TESTING_dataset_single_eye.csv'
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.8, 0.1, 0.1
NUM_CLASSES = 8

In [9]:
class ODIRDataset(Dataset) :
    def __init__(self, X, Y, IMG_FOLDER, HIST_IMG_FOLDER, transform = None) :
        '''
        id : list of samples ids as string
        '''
        self.images = X
        self.labels = Y
        self.image_dir = IMG_FOLDER
        self.hist_image_dir = HIST_IMG_FOLDER
        self.transform = transform


        t = T.Resize((299,299),interpolation=torchvision.transforms.InterpolationMode.NEAREST)
        self.images = [t(torchvision.io.read_image(os.path.join(IMG_FOLDER, x))) for x in X]
        self.hist_images = [t(torchvision.io.read_image(os.path.join(HIST_IMG_FOLDER, x))) for x in X]
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        
        #img_path = os.path.join(self.image_dir, self.images[idx])
        #hist_img_path = os.path.join(self.hist_image_dir, self.images[idx])
        
        #image = Image.open(img_path).convert("RGB")
        #hist_image = Image.open(hist_img_path)
        image = self.images[idx]
        hist_image = self.hist_images[idx]

        labels = torch.Tensor(self.labels[idx]).long()
        # t = T.Resize((299,299),interpolation=torchvision.transforms.InterpolationMode.NEAREST)
        # image = t(image)#.long()
        # hist_image = t(hist_image)
        # image = image.numpy()
        # hist_image = hist_image.numpy()
        if(self.transform != None) :
            image = image.numpy()
            hist_image = hist_image.numpy()
            image = np.transpose(image, (1,2,0))
            image = self.transform(image = image)['image']
            image = np.transpose(image, (2,0,1))
            
            hist_image = np.transpose(hist_image, (1,2,0))
            hist_image = self.transform(image = hist_image)['image']
            hist_image = np.transpose(hist_image, (2,0,1))
            image=torch.tensor(image)
            hist_image=torch.tensor(hist_image)

        return image, hist_image, labels

In [21]:
def load_model(path,num_classes=8,device="cpu"):
    model = torchvision.models.efficientnet_b3(weights =  torchvision.models.EfficientNet_B3_Weights.DEFAULT).to(device)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = torch.nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)
    model.classifier[0] = torch.nn.Dropout(p=0.5, inplace=True)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

def load_grey_model(path,num_classes=8,device="cpu"):
    model = torchvision.models.efficientnet_b3(weights =  torchvision.models.EfficientNet_B3_Weights.DEFAULT).to(device)
    num_ftrs = model.classifier[1].in_features
    model.features[0][0] = torch.nn.Conv2d(1, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    model.classifier[1] = torch.nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)
    model.classifier[0] = torch.nn.Dropout(p=0.5, inplace=True)
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    return model

def predictions(scores) :
    
    B = scores.shape[0]
    predictions = torch.empty(B, NUM_CLASSES)
    
    for i in range(B) :
      pred = torch.empty(NUM_CLASSES,)
      for j in range(NUM_CLASSES) : 
        if(scores[i][j] > 0.5) :
          pred[j] = 1
        else :
          pred[j] = 0
      predictions[i] = pred

    return predictions

In [28]:
class combined_efficientNet():    
    def __init__(self, normal_path, grey_path) :
        self.normal_efficientNet = load_model(normal_path,num_classes=8,device="cuda")
        self.grey_efficientNet = load_grey_model(grey_path,num_classes=8,device="cuda")
        
    def test_sample(self, sample, sample_hist):
        output_normal = torch.sigmoid(self.normal_efficientNet(sample))
        output_grey = torch.sigmoid(self.grey_efficientNet(sample_hist))
        outputs = torch.add(output_normal, output_grey)
        outputs = torch.div(outputs, 2)
        return outputs

    def test(self, test_loader):
        self.normal_efficientNet.eval()
        self.grey_efficientNet.eval()
        test_loss = 0
        test_acc  = 0
        AVERAGING = 'micro'
        PREC = torchmetrics.classification.MultilabelPrecision(8, average = AVERAGING)#, validate_args = False)
        ACC = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING)#, validate_args = False)
        REC = torchmetrics.classification.MultilabelRecall(8, average = AVERAGING)#, validate_args = False)
        F1_SCORE = torchmetrics.classification.MultilabelF1Score(8, average = AVERAGING)#, validate_args = False)
        F_BETA_SCORE = torchmetrics.classification.MultilabelFBetaScore(beta = 0.8, num_classes = 8, num_labels = 8, average = AVERAGING)#, validate_args = False)
        KAPPA = torchmetrics.classification.MulticlassCohenKappa(8)#, validate_args = False)
        AUC = torchmetrics.classification.MultilabelAUROC(8, average = AVERAGING)#, validate_args = False)

        for batch_idx, (data, data_hist, targets) in enumerate((test_dataloader_preheld)):
            data = data.to(device).float()
            data_hist = data_hist.float().to(device)
            targets = targets.to(device)
            with torch.no_grad():
                scores = self.test_sample(data, data_hist)
            loss = criterion(scores, targets.float())
            test_loss+= loss.item()
            predicted = predictions(scores).to(device)
            test_acc+= (torch.sum(predicted == targets)/(BATCH_SIZE*8))

            predicted = predicted.to('cpu')
            targets = targets.to('cpu')
            PREC(predicted, targets)
            ACC(predicted, targets)
            REC(predicted, targets)
            F1_SCORE(predicted, targets)
            F_BETA_SCORE(predicted, targets)
            KAPPA(predicted, targets)
            AUC(predicted, targets)


        add_prec = PREC.compute()
        add_acc = ACC.compute()
        add_rec = REC.compute()
        add_f1 = F1_SCORE.compute()
        add_fbeta = F_BETA_SCORE.compute()
        add_kappa = KAPPA.compute()
        add_auc = AUC.compute()

        avg_test_loss = test_loss/len(test_dataloader_preheld)
        avg_test_acc  = test_acc /len(test_dataloader_preheld)

        print("Acc: {:3f}\nPrec: {:3f}\nRecall: {:.3f}\nF1-score: {:.3f}\nF-Beta-score: {:.3f}\nKappa: {:.3f}\nAUC: {:.3f}".format(add_acc, add_prec,add_rec, add_f1, add_fbeta, add_kappa, add_auc))
        torch.cuda.empty_cache()

In [29]:
torch.cuda.empty_cache()

In [30]:
weight_class = torch.tensor([1,1.2,1.5,1.5,1.5,1.5, 1.5, 1.2]).to(device)
criterion = torch.nn.BCELoss(weight_class)

In [31]:
csv = pd.read_csv(CSV_PATH)
print(CSV_PATH)

csv = csv[csv['NOT DECISIVE'] == 0]

csv['img_exists'] = csv['Image'].apply(lambda x: os.path.isfile(IMG_FOLDER + "/" + x))

# drop the rows for which the file does not exist
csv = csv[csv['img_exists']]

csv.drop(columns = ['ID', 'eye', 'Patient Age',	'Patient Sex', 'NOT DECISIVE', 'img_exists'], inplace = True)
csv

../Datasets/ocular-disease-recognition-odir5k/dataset_single_eye.csv


,Image,Keywords,N,D,G,C,A,H,M,O
0,970_right.jpg,cataract,0,0,0,1,0,0,0,0
1,127_left.jpg,proliferative diabetic retinopathy，hypertensiv...,0,1,0,0,0,0,0,0
2,850_right.jpg,macular epiretinal membrane，moderate non proli...,0,1,0,0,0,0,0,0
3,37_right.jpg,normal fundus,1,0,0,0,0,0,0,0
4,4421_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5733,199_left.jpg,branch retinal vein occlusion,0,0,0,0,0,0,0,1
5734,516_right.jpg,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
5735,4603_left.jpg,severe nonproliferative retinopathy,0,1,0,0,0,0,0,0
5736,2132_right.jpg,normal fundus,1,0,0,0,0,0,0,0


In [32]:
X = csv['Image'].to_numpy()
Y = csv.drop(['Image', 'Keywords'], axis = 1).to_numpy()

print(X.shape)
print(Y.shape)

(5738,)
(5738, 8)


In [33]:
test_dataset = ODIRDataset(X, Y, IMG_FOLDER, HIST_IMG_FOLDER, transform = None)
test_dataloader_preheld = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [34]:
EfficientNet = combined_efficientNet(normal_eff_model, grey_eff_model)

In [36]:
EfficientNet.test(test_dataloader_preheld)

Acc: 0.929461
Prec: 0.802517
Recall: 0.578
F1-score: 0.672
F-Beta-score: 0.697
Kappa: 0.634
AUC: 0.779


## Metrics (With Flipping):
|Metric(micro)|Value|-|Metric(weighted)|Value|
|:-----------:|-----|-|----------------|-----|
|Acc          |0.874|-|Acc             |0.707|
|Prec:        |0.370|-|Prec:           |0.303|
|Recall       |0.005|-|Recall          |0.005|
|F1-score     |0.010|-|F1-score        |0.010|
|F-Beta-score |0.013|-|F-Beta-score    |0.013|
|Kappa        |0.007|-|Kappa           |0.007|
|AUC          |0.502|-|AUC             |0.501|

## Metrics (No Flipping)

|Metric(micro)|Value|-|Metric(weighted)|Value|
|:-----------:|-----|-|----------------|-----|
|Acc          |0.929|-|Acc             |0.852|
|Prec:        |0.802|-|Prec:           |0.718|
|Recall       |0.578|-|Recall          |0.578|
|F1-score     |0.672|-|F1-score        |0.624|
|F-Beta-score |0.697|-|F-Beta-score    |0.638|
|Kappa        |0.634|-|Kappa           |0.634|
|AUC          |0.779|-|AUC             |0.746|

Preprocessing = False

Acc: 0.929461
Prec: 0.802517
Recall: 0.578
F1-score: 0.672
F-Beta-score: 0.697
Kappa: 0.634
AUC: 0.779

Preprocessed = True

Acc: 0.874543
Prec: 0.370370
Recall: 0.005
F1-score: 0.010
F-Beta-score: 0.013
Kappa: 0.007
AUC: 0.502